In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Clustering

In [3]:
features = pd.read_csv('item_features.csv')

In [4]:
features

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75
...,...,...,...
471746,28143,68,351
471747,28143,55,390
471748,28143,11,109
471749,28143,73,91


In [5]:
n_pca_comp = 16
number_of_clusters = 4

In [6]:
cluster_assign = pd.read_csv('cluster_assign.csv')
cluster_assign = cluster_assign[['item_id2', 'Cluster_pred']]
cluster_assign

,item_id2,Cluster_pred
0,2,4
1,3,0
2,4,0
3,7,1
4,8,0
...,...,...
23686,28139,0
23687,28140,3
23688,28141,0
23689,28142,1


# Feature Engineering

- First seen product
- Instant when the first product was seen
- Last seen product
- Instant when the last product was seen
- Total session duration
- Mean time spent in each product
- Time of the day when the session started

In [7]:
df_sessions = pd.read_csv('train_sessions.csv')

In [8]:
df_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [9]:
session_sorted = df_sessions.sort_values(by=['date'])

# First date of session
begin_df = session_sorted.groupby('session_id')['date'].first().rename('start_date')

# First product of session
first_prod = session_sorted.groupby('session_id')['item_id'].first().rename('first_prod')

# Number of products seen in the session
prod_count = session_sorted.groupby('session_id')['item_id'].count().rename('prod_count')

# Last date of the session
end_df = session_sorted.groupby('session_id')['date'].last().rename('end_date')

# Last product of the session
last_prod = session_sorted.groupby('session_id')['item_id'].last().rename('last_prod')

# most common category in these session
join_cat = session_sorted.merge(features, on='item_id', how='left').sort_values(by=['feature_category_id'])
common_cat = join_cat.groupby(['session_id'])['feature_category_id'].value_counts().rename('most_common_cat')
common_cat_df = common_cat.to_frame(name = None).reset_index()
common_cat = common_cat_df.groupby(['session_id'])['feature_category_id'].first().rename('most_common_cat')

# counting number of unique categories for session
count_num_unique_cat = common_cat_df.groupby(['session_id'])['feature_category_id'].count().rename('count_num_unique_cat')


# Concat the datasets
times_df = pd.concat([begin_df, end_df, first_prod, last_prod, 
                      prod_count, common_cat, count_num_unique_cat
                     ],axis=1)

# Proper time format
times_df['start_date'] = pd.to_datetime(times_df['start_date'])
times_df['end_date'] = pd.to_datetime(times_df['end_date'])

# Calculatind the date difference
times_df['time_diff'] = (times_df.end_date-times_df.start_date).astype('timedelta64[s]')
times_df['time_diff'] = times_df['time_diff'].astype(int)

# Calculating the time per product
times_df['time_per_prod'] = (times_df['time_diff']/times_df['prod_count']).astype(int)

# Adding the time of the day
mask=(times_df.start_date.dt.hour>=0) & (times_df.start_date.dt.hour<7)
times_df.loc[mask,'time_first_prod'] = 'madrugada'
mask=(times_df.start_date.dt.hour>=7) & (times_df.start_date.dt.hour<12)
times_df.loc[mask,'time_first_prod'] = 'día'
mask=(times_df.start_date.dt.hour>=12) & (times_df.start_date.dt.hour<18)
times_df.loc[mask,'time_first_prod'] = 'tarde'
mask=(times_df.start_date.dt.hour>=18) & (times_df.start_date.dt.hour<24)
times_df.loc[mask,'time_first_prod'] = 'noche'


In [10]:
sessions=times_df
sessions

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod
session_id,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde
...,...,...,...,...,...,...,...,...,...,...
4439986,2021-05-13 11:46:56.606,2021-05-13 11:55:50.151,23502,27733,6,4,18,533,88,día
4439990,2020-08-22 11:38:48.785,2020-08-22 12:36:27.326,22093,22551,11,4,46,3458,314,día
4439994,2020-11-27 20:08:37.262,2020-11-27 20:08:37.262,25357,25357,1,3,25,0,0,noche


# Purchases

In [11]:
purchases = pd.read_csv('train_purchases.csv').set_index('session_id')
purchases['date']=pd.to_datetime(purchases['date'])
purchases.rename(columns={'item_id':'item_purch','date':'purch_date'}, inplace=True)
purchases.head()

,item_purch,purch_date
session_id,,
3,15085,2020-12-18 21:26:47.986
13,18626,2020-03-13 19:36:15.507
18,24911,2020-08-26 19:20:32.049
19,12534,2020-11-02 17:16:45.920
24,13226,2020-02-26 18:27:44.114


In [12]:
purchases.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 3 to 4440001
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   item_purch  1000000 non-null  int64         
 1   purch_date  1000000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 22.9 MB


# Features
- Tabla con sesiones y veces que se ha visto cada clúster

In [13]:
item_clust = cluster_assign[['item_id2', 'Cluster_pred']].set_index('item_id2')
item_clust

,Cluster_pred
item_id2,
2,4
3,0
4,0
7,1
8,0
...,...
28139,0
28140,3
28141,0


In [14]:
df_sess_clust=df_sessions.merge(item_clust, left_on='item_id', right_index=True)
df_sess_clust

,session_id,item_id,date,Cluster_pred
0,3,9655,2020-12-18 21:25:00.373,3
1,3,9655,2020-12-18 21:19:48.093,3
103605,97085,9655,2020-12-27 20:22:13.011,3
162423,152155,9655,2020-11-19 16:26:18.468,3
225780,210806,9655,2021-01-02 05:37:37.183,3
...,...,...,...,...
4737630,4434058,3416,2020-11-14 23:33:46.655,0
4737631,4434058,15964,2020-11-14 23:33:58.652,0
4739963,4436170,27863,2020-07-20 06:36:30.611,1
4741053,4437232,7735,2020-01-02 20:07:06.367,0


In [15]:
df_clust_count = df_sess_clust.groupby(['session_id', 'Cluster_pred'])['session_id'].aggregate('count').unstack().fillna(0)


In [16]:
df_clust_count

Cluster_pred,0,1,2,3,4
session_id,,,,,
3,0.0,0.0,0.0,2.0,0.0
13,0.0,0.0,0.0,0.0,1.0
18,3.0,0.0,0.0,0.0,0.0
19,17.0,0.0,0.0,0.0,0.0
24,0.0,3.0,2.0,1.0,3.0
...,...,...,...,...,...
4439986,0.0,0.0,0.0,0.0,6.0
4439990,8.0,0.0,0.0,1.0,2.0
4439994,1.0,0.0,0.0,0.0,0.0


What is the most seen cluster of each session?

In [17]:
df_clust_count["most_seen_cluster"] = df_clust_count.idxmax(axis=1)
df_clust_count

Cluster_pred,0,1,2,3,4,most_seen_cluster
session_id,,,,,,
3,0.0,0.0,0.0,2.0,0.0,3
13,0.0,0.0,0.0,0.0,1.0,4
18,3.0,0.0,0.0,0.0,0.0,0
19,17.0,0.0,0.0,0.0,0.0,0
24,0.0,3.0,2.0,1.0,3.0,1
...,...,...,...,...,...,...
4439986,0.0,0.0,0.0,0.0,6.0,4
4439990,8.0,0.0,0.0,1.0,2.0,0
4439994,1.0,0.0,0.0,0.0,0.0,0


# Combine all datasets

In [18]:
df=pd.concat([sessions, purchases, df_clust_count], axis=1)
df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,4,most_seen_cluster
session_id,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,0.0,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,0.0,0.0,1.0,4
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,3.0,0.0,0.0,0.0,0.0,0
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,17.0,0.0,0.0,0.0,0.0,0
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,0.0,3.0,2.0,1.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439986,2021-05-13 11:46:56.606,2021-05-13 11:55:50.151,23502,27733,6,4,18,533,88,día,2915,2021-05-13 11:56:37.464,0.0,0.0,0.0,0.0,6.0,4
4439990,2020-08-22 11:38:48.785,2020-08-22 12:36:27.326,22093,22551,11,4,46,3458,314,día,8786,2020-08-22 14:28:22.382,8.0,0.0,0.0,1.0,2.0,0
4439994,2020-11-27 20:08:37.262,2020-11-27 20:08:37.262,25357,25357,1,3,25,0,0,noche,21630,2020-11-27 20:10:28.961,1.0,0.0,0.0,0.0,0.0,0


What was the cluster of the first and the last seen item?

In [19]:
df = df.reset_index().merge(item_clust, how='left', left_on='first_prod', right_on=item_clust.index).set_index('session_id')
df = df.rename(columns = {'Cluster_pred':'first_item_cluster'})
df = df.reset_index().merge(item_clust, how='left', left_on='last_prod', right_on=item_clust.index).set_index('session_id')
df = df.rename(columns = {'Cluster_pred':'last_item_cluster'})

In [20]:
df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,4,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,0.0,3,3,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,0.0,0.0,1.0,4,4,4
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,3.0,0.0,0.0,0.0,0.0,0,0,0
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,17.0,0.0,0.0,0.0,0.0,0,0,0
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,0.0,3.0,2.0,1.0,3.0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439986,2021-05-13 11:46:56.606,2021-05-13 11:55:50.151,23502,27733,6,4,18,533,88,día,2915,2021-05-13 11:56:37.464,0.0,0.0,0.0,0.0,6.0,4,4,4
4439990,2020-08-22 11:38:48.785,2020-08-22 12:36:27.326,22093,22551,11,4,46,3458,314,día,8786,2020-08-22 14:28:22.382,8.0,0.0,0.0,1.0,2.0,0,0,0
4439994,2020-11-27 20:08:37.262,2020-11-27 20:08:37.262,25357,25357,1,3,25,0,0,noche,21630,2020-11-27 20:10:28.961,1.0,0.0,0.0,0.0,0.0,0,0,0


In order to reduce the training time, we will need to reduce the number of sessions to 112055 (id of the session must be less or equal than 500000).

In [21]:
df = df[df.index<=100000]

df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,4,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,0.0,3,3,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,0.0,0.0,1.0,4,4,4
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,3.0,0.0,0.0,0.0,0.0,0,0,0
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,17.0,0.0,0.0,0.0,0.0,0,0,0
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,0.0,3.0,2.0,1.0,3.0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499990,2020-08-23 08:18:20.974,2020-08-23 08:19:17.678,17186,653,2,4,28,56,28,día,16257,2020-08-23 08:19:35.204,2.0,0.0,0.0,0.0,0.0,0,0,0
499991,2021-01-15 20:29:29.571,2021-01-15 20:29:29.571,27519,27519,1,3,25,0,0,noche,2447,2021-01-15 20:33:33.934,1.0,0.0,0.0,0.0,0.0,0,0,0
499993,2020-04-18 09:10:46.566,2020-04-18 09:14:51.287,27585,14999,4,47,37,244,61,día,27197,2020-04-18 09:16:25.158,1.0,0.0,0.0,3.0,0.0,3,3,3


In [22]:
df.to_csv('dataset_merge.csv')

### How many type of items are seen in each session?

This gives us valuable insights on whether the clustering is giving us logical information. If the clustering has been made correctly, it would be normal to expect that in the majority of sessions only one type of products has been seen. If the clustering returned random results, not many sessions would have only one type of product seen because the products would have been assigned to clusters randomly, and each session would normally have more than one type of cluster.

In [23]:
df_clusts = df[range(0,number_of_clusters)]
df_clusts = df_clusts.replace(0, np.nan)
df_clusts["distinct"] = df_clusts.count(1)
#df_clusts.head()
item_counts = df_clusts["distinct"].value_counts()
print(item_counts)

1    61588
0    26772
2    18754
3     4388
4      553
Name: distinct, dtype: int64


In [24]:
df_clusts = df[range(0,number_of_clusters)]
df_clusts = df_clusts.replace(0, np.nan)
df_clusts["sum"] = df_clusts.sum(axis=1)
df_clusts = df_clusts[df_clusts["sum"] > 4.0]
df_clusts["distinct"] = df_clusts.count(1)
df_clusts.head()
item_counts = df_clusts["distinct"].value_counts()
print(item_counts)

2    10671
3    10560
4     3791
5      544
Name: distinct, dtype: int64


## Divide into train - validation

In [25]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=40)
len(train_set),len(test_set)

(89644, 22411)

In [26]:
cat_vars = ["first_prod", "last_prod", "time_first_prod", "most_common_cat", "most_seen_cluster", "first_item_cluster", "last_item_cluster"]
num_vars = ["prod_count", "time_diff", "time_per_prod", "count_num_unique_cat"] + list(range(0,number_of_clusters))

## Prepare the data for Machine Learning algorithms

In [27]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(handle_unknown='ignore')
train_set_1hot = cat_encoder.fit_transform(train_set.fillna('Unknown'))

In [28]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [29]:
train_set_num = train_set[num_vars]

In [30]:
imputer.fit(train_set_num)
imputer.statistics_

array([  3., 108.,  30.,  25.,   1.,   0.,   0.,   0.])

In [31]:
X = imputer.transform(train_set_num)

In [32]:
train_set_tr = pd.DataFrame(X, columns=train_set_num.columns,
                          index=train_set.index)

In [33]:
train_set_tr

,prod_count,time_diff,time_per_prod,count_num_unique_cat,0,1,2,3
session_id,,,,,,,,
480062,15.0,737.0,49.0,36.0,5.0,6.0,0.0,0.0
266546,9.0,428.0,47.0,35.0,6.0,2.0,1.0,0.0
386792,11.0,5851.0,531.0,36.0,2.0,0.0,0.0,0.0
369149,7.0,706.0,100.0,31.0,1.0,0.0,0.0,0.0
332676,1.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
252371,5.0,191.0,38.0,38.0,3.0,0.0,0.0,0.0
416048,5.0,108.0,21.0,30.0,1.0,0.0,0.0,0.0
357865,1.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0


### Categorical data

Now let's preprocess the categorical input:

In [34]:
train_set_cat = train_set[cat_vars]
train_set_cat.head(10)

,first_prod,last_prod,time_first_prod,most_common_cat,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,
480062,7327,6865,noche,30,1,4,1
266546,24700,8111,tarde,7,0,0,0
386792,6113,4286,noche,4,4,4,4
369149,23502,23028,tarde,4,4,4,0
332676,10498,10498,tarde,4,4,4,4
390994,11915,23335,tarde,3,0,0,0
306659,6372,6372,tarde,30,2,2,2
314156,12179,12179,día,4,4,4,4
199684,21786,15324,noche,4,2,2,2


In [35]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
train_set_cat_encoded = ordinal_encoder.fit_transform(train_set_cat)
train_set_cat_encoded[:10]

array([[3.7660e+03, 3.6540e+03, 2.0000e+00, 1.0000e+01, 1.0000e+00,
        4.0000e+00, 1.0000e+00],
       [1.2630e+04, 4.3530e+03, 3.0000e+00, 6.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [3.1390e+03, 2.2810e+03, 2.0000e+00, 3.0000e+00, 4.0000e+00,
        4.0000e+00, 4.0000e+00],
       [1.2026e+04, 1.2255e+04, 3.0000e+00, 3.0000e+00, 4.0000e+00,
        4.0000e+00, 0.0000e+00],
       [5.4010e+03, 5.6520e+03, 3.0000e+00, 3.0000e+00, 4.0000e+00,
        4.0000e+00, 4.0000e+00],
       [6.1310e+03, 1.2409e+04, 3.0000e+00, 2.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [3.2820e+03, 3.4070e+03, 3.0000e+00, 1.0000e+01, 2.0000e+00,
        2.0000e+00, 2.0000e+00],
       [6.2610e+03, 6.5360e+03, 0.0000e+00, 3.0000e+00, 4.0000e+00,
        4.0000e+00, 4.0000e+00],
       [1.1168e+04, 8.1560e+03, 2.0000e+00, 3.0000e+00, 2.0000e+00,
        2.0000e+00, 2.0000e+00],
       [3.2380e+03, 1.0566e+04, 3.0000e+00, 1.0000e+00, 3.0000e+00,
        3.0000e+00, 3.0

In [36]:
cat_vars

['first_prod',
 'last_prod',
 'time_first_prod',
 'most_common_cat',
 'most_seen_cluster',
 'first_item_cluster',
 'last_item_cluster']

In [37]:
from sklearn.preprocessing import OneHotEncoder
items=cluster_assign.item_id2.to_list()
cats=list(range(1,74))
clusts=list(range(0,number_of_clusters))
times=['día','madrugada','tarde','noche']
cat_encoder = OneHotEncoder(categories=[items,items,times,cats,clusts,clusts,clusts], handle_unknown='ignore')
train_set_cat_1hot = cat_encoder.fit_transform(train_set_cat)
train_set_cat_1hot

<89644x47471 sparse matrix of type '<class 'numpy.float64'>'
	with 541924 stored elements in Compressed Sparse Row format>

In [38]:
train_set_cat_1hot.toarray()

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

train_set_num_tr = num_pipeline.fit_transform(train_set_num)

In [40]:
cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="constant",fill_value='Unknown')),
        ('one_hot_encoder', OneHotEncoder(categories=[items,items,times,cats,clusts,clusts,clusts], handle_unknown='ignore')),
    ])

train_set_cat_tr = cat_pipeline.fit_transform(train_set_cat)

In [41]:
from sklearn.compose import ColumnTransformer

num_attribs1 = train_set_num.columns
cat_attribs = train_set_cat.columns


full_pipeline_train = ColumnTransformer([
        ("num1", num_pipeline, num_attribs1),
        ("cat", cat_pipeline, cat_attribs),
    ])

train_set_prepared = full_pipeline_train.fit_transform(train_set)
train_set_prepared

<89644x47479 sparse matrix of type '<class 'numpy.float64'>'
	with 1259076 stored elements in Compressed Sparse Row format>

In [42]:
type(train_set_prepared)

scipy.sparse.csr.csr_matrix

## Training and model selection

### SGD Classifier

In [43]:
from sklearn.linear_model import SGDClassifier

#sgd_class = SGDClassifier(loss='modified_huber', random_state=42)
#sgd_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(sgd_class, open('models/sgd_model.sav', 'wb'))

### Decision Tree

In [44]:
from sklearn.tree import DecisionTreeClassifier

#tree_class = DecisionTreeClassifier(random_state=42)
#tree_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(tree_class, open('models/tree_model.sav', 'wb'))

KeyboardInterrupt: 

### K-Nearest Neighbour

In [ ]:
from sklearn import neighbors

#n_neighbors = 3
#knn_class = neighbors.KNeighborsClassifier(n_neighbors)
#knn_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(knn_class, open('models/knn_model.sav', 'wb'))

### SVC

In [ ]:
from sklearn import svm

#SVC_class = svm.SVC(kernel='linear', probability=True)
#SVC_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(SVC_class, open('models/svc_model.sav', 'wb'))

### Random Forest Classifier

In [45]:
from sklearn.ensemble import RandomForestClassifier

forest_class = RandomForestClassifier(n_estimators=20, random_state=42)
forest_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(forest_class, open('models/random_model.sav', 'wb'))

In [ ]:
from sklearn.feature_selection import SelectFromModel

#sel = SelectFromModel(RandomForestClassifier(n_estimators = 20))
#sel.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())
#sel.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#sel.get_support()

KeyboardInterrupt: 

## Model testing

#### Test each model using the mean reciprocal rank method. That is the one that is used in the RecSys Challenge competition. This will help us choose which model we are going to end up using. 

In [ ]:
test_set_prepared = full_pipeline_train.fit_transform(test_set)

In [ ]:
def mean_reciprocal_rank(model, set_to_test, rr = 0, cont = 0):

    predictions = model.predict_proba(set_to_test)

    pred_df_test = pd.DataFrame(predictions)
    pred_df_test.columns = model.classes_

    pred_df_test["session_id"] = test_set.index
    pred_df_test = pred_df_test.merge(df[["item_purch"]], how='inner', on='session_id')
    pred_df_test = pred_df_test[["session_id", "item_purch"] + list(pred_df_test.columns[:-2])]

    for index, row in pred_df_test.iterrows():
        item_purch_act = int(row.iloc[1])
        row = row.iloc[2:]
        row_sorted = row.sort_values(ascending=False)
        items = row_sorted.index.to_list()
        if item_purch_act in items:
            rank = int(items.index(item_purch_act)) + 1
            if rank <= 100:
                rr += 1/rank
        cont += 1

    mrr = rr/cont
    print("The mean reciprocal rank for the " + str(model) + " is " + str(mrr))

In [ ]:
#SVC_class = pickle.load(open('models/svc_model.sav', 'rb'))
#mean_reciprocal_rank(sgd_class, test_set_prepared)
#mean_reciprocal_rank(knn_class, test_set_prepared)
#mean_reciprocal_rank(SVC_class, test_set_prepared)
#mean_reciprocal_rank(tree_class, test_set_prepared)
mean_reciprocal_rank(forest_class, test_set_prepared)

The mean reciprocal rank for the KNeighborsClassifier(n_neighbors=3) is 0.018015712930821284
The mean reciprocal rank for the DecisionTreeClassifier(random_state=42) is 0.012818645361358351
The mean reciprocal rank for the RandomForestClassifier(max_depth=5, max_features=4, n_estimators=10,
                       random_state=42) is 0.013266096283686161


Classifier | Number of Sessions | Mean Reciprocal Rank | Training Time |
| --- | --- | --- | --- |
SGD | 50000 | 0.001205949176443282 | 6s 900ms |
Decision Tree | 50000 | 0.011661608415718349 | 300ms |
KNN | 50000 | 0.0022989210718511836 | 200ms |
SVC | 50000 | 0.012714121903862513 | 2min 51s 800ms |
Random Forest | 50000 | 0.011327447878240757 | 100ms |

Based on the results we got for each classifier, we can see that the ones that got the best results were the Decision Tree, the Random Forest and the SVC classifier. We are going to have to rule out the SVC classifier, due to its long training time. Between the Decision Tree and the Random Forest classifier, since they have very similar MRR, we are going to choose the one with the shortest training time. 

In conclusion, the Random Forest is the classifier we will use.

## Best Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8], 'max_depth':[3,5,7,10]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]


# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_class, param_grid, cv=5,
                           scoring='neg_root_mean_squared_error',
                           return_train_score=True)
#grid_search.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=20, random_state=42),
             param_grid=[{'max_depth': [3, 5, 7, 10],
                          'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [ ]:
print("the best parameters are:")
#print(grid_search.best_params_)

the best parameters are:
{'max_depth': 5, 'max_features': 4, 'n_estimators': 10}


## Feature Importance

### Feature importance based on mean decrease in impurity

In [ ]:
import time
import numpy as np

start_time = time.time()
importances = forest_class.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest_class.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

Elapsed time to compute the importances: 0.001 seconds


In [ ]:
import seaborn as sns

def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

plot_feature_importance(forest_class.feature_importances_,cat_vars+num_vars,'RANDOM FOREST')


ValueError: All arrays must be of the same length

In [ ]:
len(forest_class.feature_importances_)

47478

In [ ]:
importances

NameError: name 'importances' is not defined

In [ ]:
import pandas as pd

vars = cat_vars + num_vars
vars = np.array(vars)
forest_importances = pd.Series(importances, index= vars)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

ValueError: Length of values (4) does not match length of index (15)

## Generate TEST Submission

In [ ]:
df_leader = pd.read_csv('test_leaderboard_sessions.csv')

In [ ]:
leader_session_sorted = df_leader.sort_values(by=['date'])

# First date of session
leader_begin_df = leader_session_sorted.groupby('session_id')['date'].first().rename('start_date')

# First product of session
leader_first_prod=leader_session_sorted.groupby('session_id')['item_id'].first().rename('first_prod')

# Number of products seen in the session
leader_prod_count=leader_session_sorted.groupby('session_id')['item_id'].count().rename('prod_count')

# Last date of the session
leader_end_df = leader_session_sorted.groupby('session_id')['date'].last().rename('end_date')

# Last product of the session
leader_last_prod=leader_session_sorted.groupby('session_id')['item_id'].last().rename('last_prod')

# most common category in these session
leader_join_cat = leader_session_sorted.merge(features, on='item_id', how='left').sort_values(by=['feature_category_id'])
leader_common_cat = leader_join_cat.groupby(['session_id'])['feature_category_id'].value_counts().rename('most_common_cat')
leader_common_cat_df = leader_common_cat.to_frame(name = None).reset_index()
leader_common_cat = leader_common_cat_df.groupby(['session_id'])['feature_category_id'].first().rename('most_common_cat')

# counting number of unique categories for session
leader_count_num_unique_cat = leader_common_cat_df.groupby(['session_id'])['feature_category_id'].count().rename('count_num_unique_cat')


# Concat the datasets
leader_times_df = pd.concat([leader_begin_df, leader_end_df, leader_first_prod, leader_last_prod, 
                      leader_prod_count, leader_common_cat, leader_count_num_unique_cat
                     ],axis=1)

# Proper time format
leader_times_df['start_date']=pd.to_datetime(leader_times_df['start_date'])
leader_times_df['end_date']=pd.to_datetime(leader_times_df['end_date'])

# Calculatind the date difference
leader_times_df['time_diff']=(leader_times_df.end_date-leader_times_df.start_date).astype('timedelta64[s]')

# Calculating the time per product
leader_times_df['time_per_prod']=leader_times_df['time_diff']/leader_times_df['prod_count']

# Adding the time of the day
mask=(leader_times_df.start_date.dt.hour>=0) & (leader_times_df.start_date.dt.hour<7)
leader_times_df.loc[mask,'time_first_prod']='madrugada'
mask=(leader_times_df.start_date.dt.hour>=7) & (leader_times_df.start_date.dt.hour<12)
leader_times_df.loc[mask,'time_first_prod']='día'
mask=(leader_times_df.start_date.dt.hour>=12) & (leader_times_df.start_date.dt.hour<18)
leader_times_df.loc[mask,'time_first_prod']='tarde'
mask=(leader_times_df.start_date.dt.hour>=18) & (leader_times_df.start_date.dt.hour<24)
leader_times_df.loc[mask,'time_first_prod']='noche'

In [ ]:
leader_times_df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod
session_id,,,,,,,,,,
26,2021-06-16 09:53:54.158,2021-06-16 09:53:54.158,19185,19185,1,3,24,0.0,0.000000,día
200,2021-06-25 12:23:40.811,2021-06-25 12:24:50.692,17089,4758,4,30,19,69.0,17.250000,tarde
205,2021-06-11 00:28:07.058,2021-06-11 00:28:07.058,8194,8194,1,3,25,0.0,0.000000,madrugada
495,2021-06-14 22:13:06.741,2021-06-14 22:13:06.741,6853,6853,1,4,20,0.0,0.000000,noche
521,2021-06-19 13:50:03.090,2021-06-19 13:50:03.090,26471,26471,1,3,25,0.0,0.000000,tarde
...,...,...,...,...,...,...,...,...,...,...
4439446,2021-06-11 17:16:12.243,2021-06-11 17:17:11.565,20770,21396,3,4,19,59.0,19.666667,tarde
4439458,2021-06-09 04:52:02.785,2021-06-09 04:54:45.378,11715,26499,2,4,29,162.0,81.000000,madrugada
4439550,2021-06-02 17:42:40.481,2021-06-02 17:42:40.481,19086,19086,1,28,18,0.0,0.000000,tarde


In [ ]:
df_sess_clust_leader=df_leader.merge(item_clust, left_on='item_id', right_index=True)
df_sess_clust_leader

,session_id,item_id,date,Cluster_pred
0,26,19185,2021-06-16 09:53:54.158,0
13240,239993,19185,2021-06-21 11:22:20.096,0
34682,661695,19185,2021-06-27 09:31:21.179,0
36264,699226,19185,2021-06-07 09:58:16.167,0
42810,830528,19185,2021-06-14 15:26:44.282,0
...,...,...,...,...
226231,4382253,7523,2021-06-18 14:05:33.111,4
227011,4397116,6327,2021-06-23 21:59:23.999,2
227515,4408745,10278,2021-06-06 11:23:56.511,3
228684,4428903,27394,2021-06-16 17:03:01.114,0


In [ ]:
df_sess_clust

,session_id,item_id,date,Cluster_pred
0,3,9655,2020-12-18 21:25:00.373,3
1,3,9655,2020-12-18 21:19:48.093,3
103605,97085,9655,2020-12-27 20:22:13.011,3
162423,152155,9655,2020-11-19 16:26:18.468,3
225780,210806,9655,2021-01-02 05:37:37.183,3
...,...,...,...,...
4737630,4434058,3416,2020-11-14 23:33:46.655,0
4737631,4434058,15964,2020-11-14 23:33:58.652,0
4739963,4436170,27863,2020-07-20 06:36:30.611,1
4741053,4437232,7735,2020-01-02 20:07:06.367,0


In [ ]:
df_clust_count_leader = df_sess_clust_leader.groupby(['session_id', 'Cluster_pred'])['session_id'].aggregate('count').unstack().fillna(0)

In [ ]:
df_clust_count_leader["most_seen_cluster"] = df_clust_count_leader.idxmax(axis=1)

In [ ]:
df_leader=pd.concat([leader_times_df, df_clust_count_leader], axis=1)

In [ ]:
df_leader = df_leader.reset_index().merge(item_clust, how='left', left_on='first_prod', right_on=item_clust.index).set_index('session_id')
df_leader = df_leader.rename(columns = {'Cluster_pred':'first_item_cluster'})
df_leader = df_leader.reset_index().merge(item_clust, how='left', left_on='last_prod', right_on=item_clust.index).set_index('session_id')
df_leader = df_leader.rename(columns = {'Cluster_pred':'last_item_cluster'})

In [ ]:
leader_set_prepared = full_pipeline_train.fit_transform(df_leader)

In [ ]:
predictions = forest_class.predict_proba(leader_set_prepared)
#print(predictions)

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.columns = forest_class.classes_
pred_df

,11,18,26,28,32,44,47,54,74,89,...,28101,28103,28112,28114,28117,28122,28124,28133,28135,28139
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
candidate_items_df = pd.read_csv("candidate_items.csv")
columns = candidate_items_df["item_id"].to_list()
real_columns = [b for b in pred_df.columns if b in columns]

In [ ]:
pred_df = pred_df[real_columns]
pred_df["session_id"] = df_leader.index
pred_df = pred_df[["session_id"] + list(pred_df.columns[:-1])]
pred_df

/var/folders/dc/zzpshy3n5qzdd7ywk2xyx44m0000gn/T/ipykernel_47050/3076393300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df["session_id"] = df_leader.index


,session_id,26,54,93,95,107,112,150,160,192,...,27974,27991,28004,28030,28051,28078,28101,28112,28122,28133
0,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0
1,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
2,205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
3,495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0
4,521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4439446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
49996,4439458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
49997,4439550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
49998,4439653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0


In [ ]:
f = open('final_submission2.csv', 'w')
f.write("session_id,item_id,rank" + '\n')

for index, row in pred_df.iterrows():
    #print(index,row)
    session_id_act = int(row.iloc[0])
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    items = row_sorted.index.to_list()
    #print(items[0])
    #break
    for i in range(1,101):
        f.write(str(int(session_id_act)) + ',' + str(items[i-1]) + ',' + str(i) + '\n')